In [1]:
import numpy as np

# Вариант 1. Задание №1.
## Илларионов Владислав 855

## Начальные значения

In [2]:
gamma_0 = 5/3
rho_0 = 2.219e-4
U_0 = -1.587e5
P_0 = 3.7812e6

gamma_3 = 5/3
rho_3 = 2.71e-3
U_3 = 10
P_3 = 5e5

## Обозначения

In [3]:
C_0 = (gamma_0 * P_0 / rho_0) ** 0.5
C_3 = (gamma_3 * P_3 / rho_3) ** 0.5

alpha_0 = (gamma_0 + 1) / (gamma_0 - 1)
alpha_3 = (gamma_3 + 1) / (gamma_3 - 1)

X = P_3 / P_0

e_0 = 2 * C_0**2 / (gamma_0 * (gamma_0 - 1) * (U_3 - U_0)**2)
e_3 = 2 * C_3**2 / (gamma_3 * (gamma_3 - 1) * (U_3 - U_0)**2)

## Коэффициенты полинома

$$ f(Y) = a_0 Y^6 + a_1 Y^5 + a_2 Y^4 + a_3 Y^3 + a_4 Y^2 + a^5 Y + a_6, \hspace{1cm} Y = \frac{P_1}{P_0} $$

In [4]:
a_0 = (alpha_0 * e_3 - alpha_3 * X * e_0) ** 2

In [5]:
a_1 = 2 * (
    (alpha_0 * e_3 - alpha_3 * X * e_0)
    * (e_3 * (1 - 2 * alpha_0 * X) - e_0 * X * (X - 2 * alpha_3))
    - (alpha_0 * alpha_3 * X * (alpha_0 * e_3 + alpha_3 * X * e_0))
)

In [6]:
a_2 = (
    e_3**2 * (6 * alpha_0**2 * X**2 - 8 * alpha_0 * X + 1)
    - 2*e_0*e_3*X*(alpha_0*alpha_3*(X**2 + 4*X + 1) - 2*(X + 1)*(alpha_3 + alpha_0*X) + X)
    + e_0**2 * X**2 * (6 * alpha_3**2 - 8 * alpha_3 * X + X**2) + alpha_0**2 * alpha_3**2 * X**2
    - 2 * alpha_0 * X * e_3 * (alpha_0 * X - 2 * alpha_0 * alpha_3 * X + 2 * alpha_3)
    - 2 * alpha_3 * X**2 * e_0 * (alpha_3 + 2 * alpha_0 * X - 2 * alpha_0 * alpha_3)
)

In [7]:
a_3 = -2 * X * (
    2 * e_3**2 * (alpha_0**2 * X**2 - 3 * alpha_0 * X + 1)
    + e_0 * e_3 * ((alpha_3 + alpha_0 * X) * (X**2 + 4*X + 1) - 2*alpha_0*alpha_3*X*(X + 1) - 2*X*(X + 1))
    + 2 * e_0**2 * X * (X**2 - 3 * alpha_3 * X + alpha_3**2) - alpha_0 * alpha_3 * X * (alpha_0 * X + alpha_3)
    + e_3 * (alpha_0**2 * alpha_3 * X**2 - 2*X*(2 * alpha_0 * alpha_3 + alpha_0**2 * X) + 2*alpha_0*X + alpha_3)
    + e_0 * X * (alpha_0 * alpha_3**2 - 2 * alpha_3 * (alpha_3 + 2 * alpha_0 * X) + 2 * alpha_3 * X + alpha_0 * X**2)
)

In [8]:
a_4 = X**2 * (
    e_3**2 * (alpha_0**2 * X**2 - 8 * alpha_0 * X + 6)
    - 2 * e_0 * e_3 * (alpha_0 * alpha_3 * X - 2*(X + 1)*(alpha_3 + alpha_0 * X) + X**2 + 4*X + 1)
    + e_0**2 * (alpha_3**2 - 8 * alpha_3 * X + 6 * X**2) + (alpha_3**2 + 4*alpha_0*alpha_3*X + alpha_0**2 * X**2)
    - 2 * e_3 * (X*(alpha_0**2 * X + 2 * alpha_0 * alpha_3) - 2*(2*alpha_0*X + alpha_3) + 1)
    - 2 * e_0 * (alpha_3*(2*alpha_0*X + alpha_3) - 2*X*(2*alpha_3 + alpha_0*X) + X**2)
)

In [9]:
a_5 = 2 * X**3 * (
    e_3**2 * (alpha_0*X - 2) - e_0*e_3*(alpha_0*X - 2 + alpha_3 - 2*X)
    + e_0**2 * (alpha_3 - 2*X) + alpha_3 + alpha_0*X
    - e_3 * (2*alpha_0*X + alpha_3 - 2) - e_0 * (2*alpha_3 + alpha_0*X - 2*X)
)

In [10]:
a_6 = X**4 * ((e_3 - e_0)**2 + 1 - 2*(e_3 + e_0))

In [11]:
a = np.array([a_0, a_1, a_2, a_3, a_4, a_5, a_6])

## Определим область нахождения корней

Из следствия основной теоремы алгебры все корни расположены на комплексной плоскости в кольце:

$$ \dfrac{|a_6|}{|a_6| + B} \le |Y| \le 1 + \dfrac{A}{|a_0|} $$

In [12]:
A = np.absolute(a[1:]).max()
B = np.absolute(a[:-1]).max()

left = abs(a[-1]) / (abs(a[-1] + B))
right = 1 + A / a[0]

## Построим ряд Штурма

$$ f_0 = f $$

$$ f_1 = f_0' $$

$$ f_i = - Res(f_{i-2}, f_{i-1}), \hspace{1cm} i \in [2, 6] $$

In [13]:
f = [a]
f.append(np.polyder(a))
for i in range(2, a.size):
    f.append(-np.polydiv(f[i-2], f[i-1])[1])

In [14]:
left_row_t = []
right_row_t = []
for f_i in f:
    left_row_t.append(np.polyval(f_i, left))
    right_row_t.append(np.polyval(f_i, right))
left_row = np.array(left_row_t)
right_row = np.array(right_row_t)

### Считаем количества перемен знаков в рядах и находим количество корней

In [15]:
k_left = ((left_row[1:] * left_row[:-1]) < 0).sum()
k_right = ((right_row[1:] * right_row[:-1]) < 0).sum()

k = abs(k_left - k_right)

## Локализуем положительные действительные корни

Разбиваем отрезок пока не найдется k отрезков (k - количество корней из метода Штурма), на которых функция
обнуляется (меняет знак на концах)

In [16]:
N = 1000
count = 0
while count != k:
    x = np.linspace(left, right, N)
    y = np.polyval(a, x)
    indexes = ((y[1:] * y[:-1]) < 0)
    count = indexes.sum()
    N *= 10

In [17]:
loc_segments = np.vstack([x[:-1][indexes], x[1:][indexes]]).T

## Найдем корни методом дихотомии

In [18]:
accuracy = 1e-9

roots = []
for seg in loc_segments.copy():
    while (seg[1] - seg[0]) > accuracy:
        t = (seg[0] + seg[1]) / 2
        if (np.polyval(a, t) >= 0) == (np.polyval(a, seg[0]) >= 0):
            seg[0] = t
        else:
            seg[1] = t
    roots.append(t)

## Найдем $D_0$ и $D_3$

Законы сохранения массы и импульса на разрыве:

$$ \rho_1 (U_1 - D_0) = \rho_0 (U_0 - D_0)$$

$$ P_1 + \rho_1 (U_1 - D_0)^2 = P_0 + \rho_0 (U_0 - D_0)^2 $$

Выразим из первого уравнения $U_1 - D_0$:

$$ U_1 - D_0 = \frac{\rho_0}{\rho_1} (U_0 - D_0) $$

Подставляем во второе:

$$  P_1 + \frac{\rho_0^2}{\rho_1} (U_0 - D_0)^2 = P_0 + \rho_0 (U_0 - D_0)^2 $$

Теперь выражаем $U_0 - D_0$:

$$ (U_0 - D_0)^2 = \dfrac{P_1 - P_0}{\rho_0} \cdot \left[1 - \dfrac{\rho_0}{\rho_1}\right]^{-1} $$

Теперь запишем это выражение используя найденный корень $y = \frac{P_1}{P_0}$:

$$ (U_0 - D_0)^2 = \dfrac{P_0}{\rho_0} \cdot (y - 1) \cdot \dfrac{1}{1 - q} = d,$$

здесь использовано обзначение $q$:
$$ q = \frac{\rho_0}{\rho_1} = \dfrac{(\gamma_0 + 1) + (\gamma_0 - 1)y}{(\gamma_0 - 1) + (\gamma_0 + 1)y} $$

Теперь можем найти $D_0$:

$$ D_0 = U_0 \mp \sqrt{d} $$

Знак выбирается исходя из того, какое давление больше, $P_1$ или $P_0$

Аналогичным образом получается выражение для $D_3$

In [19]:
def calc_d0(y):
    q = ((gamma_0 + 1) + (gamma_0 - 1)*y) / ((gamma_0 - 1) + (gamma_0 + 1)*y)
    d = P_0 / rho_0 * (y - 1) / (1 - q)
    d0 = U_0 + d**0.5*np.sign(y - 1)
    return d0

def calc_d3(y):
    P_2 = y * P_0
    q = ((gamma_3 + 1)*P_3 + (gamma_3 - 1)*P_2) / ((gamma_3 - 1)*P_3 + (gamma_3 + 1)*P_2)
    d = (P_2 - P_3) / (rho_3 * (1 - q))
    d3 = U_3 - d**0.5*np.sign(y - 1)
    return d3

# Ответ

In [20]:
for r in roots.copy():
    print(f'root = {r} :\t D_0 = {calc_d0(r)}, \t D_3 = {calc_d3(r)}')

root = 0.1321172559642434 :	 D_0 = -251875.95805943938, 	 D_3 = 17539.616734387662
root = 0.13215649745679026 :	 D_0 = -251880.7422843586, 	 D_3 = 17541.698905446145
root = 2.6405578844612783 :	 D_0 = 97569.17875491877, 	 D_3 = -70515.97149779115
root = 6.4990853343483845 :	 D_0 = 232886.76187524368, 	 D_3 = -110227.00614296627
